In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import numpy as np


c:\users\mahmoud\desktop\deepsign2text-master\local\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(29, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer = 'adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = False,
                                   validation_split = 0.05)

test_datagen = ImageDataGenerator(rescale = 1./255)



training_set = train_datagen.flow_from_directory('data/asl_alphabet_train/',
                                                 target_size = (100, 100),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                            subset = 'training')
test_set = train_datagen.flow_from_directory('data/asl_alphabet_train/',
                                            target_size = (100, 100),
                                           batch_size = 32,
                                          class_mode = 'categorical',
                                           subset = 'validation')

Found 82650 images belonging to 29 classes.
Found 4350 images belonging to 29 classes.


In [4]:
history = model.fit_generator(training_set,
                         epochs = 30,
                         validation_data = test_set,)



Epoch 1/30
2583/2583 [==============================] - 231s 89ms/step - loss: 1.4431 - acc: 0.5609 - val_loss: 0.7494 - val_acc: 0.7474
Epoch 2/30
2583/2583 [==============================] - 203s 79ms/step - loss: 0.3492 - acc: 0.8818 - val_loss: 0.4646 - val_acc: 0.8352
Epoch 3/30
2583/2583 [==============================] - 203s 79ms/step - loss: 0.2101 - acc: 0.9297 - val_loss: 0.5233 - val_acc: 0.8154
Epoch 4/30
2583/2583 [==============================] - 202s 78ms/step - loss: 0.1590 - acc: 0.9474 - val_loss: 0.4704 - val_acc: 0.8474
Epoch 5/30
2583/2583 [==============================] - 203s 79ms/step - loss: 0.1266 - acc: 0.9581 - val_loss: 0.3780 - val_acc: 0.8795
Epoch 6/30
2583/2583 [==============================] - 202s 78ms/step - loss: 0.1129 - acc: 0.9648 - val_loss: 0.3073 - val_acc: 0.9069
Epoch 7/30
2583/2583 [==============================] - 203s 78ms/step - loss: 0.0949 - acc: 0.9695 - val_loss: 0.3133 - val_acc: 0.9122
Epoch 8/30
2583/2583 [===================

In [20]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('data/asl_alphabet_test/E/E_test.jpg', target_size = (100, 100))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices

print("Predicting this is: " + str(result))

Predicting this is: [[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]


## Saving the Model

In [ ]:
# Save the weights
model.save_weights('models/cnn/model_weights.h5')

# Save the model architecture
with open('models/cnn/model_architecture.json', 'w') as f:
    f.write(model.to_json())

## Graph the accuracy and loss

In [22]:
import matplotlib.pyplot as plt
%matplotlib
#summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

Using matplotlib backend: TkAgg


## Loading the Model

In [7]:
from keras.models import model_from_json

json_file = open('models/cnn/model_architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# loading weights
loaded_model.load_weights("models/cnn/model_weights.h5")

loaded_model.compile(optimizer = 'adam', loss='categorical_crossentropy',metrics=['accuracy'])
loaded_model.summary()

## Visualization of the network

In [4]:
from keras.utils.vis_utils import plot_model
plot_model(loaded_model, to_file="resources/cnn.png", show_shapes=True, show_layer_names=True)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

## Trying Network Evaluation and Confusion Matrix Generation (unsuccessful)

In [45]:
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import model_from_json
from keras.utils import np_utils
#json_file = open('C:/Users/Mahmoud/Desktop/DeepSign2Text-master/model_architecture.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
# load weights into new model
#loaded_model.load_weights("C:/Users/Mahmoud/Desktop/DeepSign2Text-master/model_weights.h5")
#print("Loaded model from disk")
 
# evaluate loaded model on test data
#loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
Y_pred = model.predict_generator(test_set)
y_pred = np.argmax(Y_pred, axis=1)

y_test = np_utils.to_categorical(test_set, 29)
print(y_test)
for ix in range(10):
    print(ix, confusion_matrix(np.argmax(y_test,axis=0),y_pred)[ix].sum())
cm = confusion_matrix(np.argmax(y_test,axis=0),y_pred)
print(cm)

# Visualizing of confusion matrix
import seaborn as sn
import pandas  as pd


df_cm = pd.DataFrame(cm, range(10),
                  range(10))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 12})# font size
plt.show()

MemoryError: 

In [8]:
score = model.evaluate(np.expand_dims(training_set, axis=3), test_set, batch_size=32)
print(score)

MemoryError: 